<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/LEJEPA_IJEPA_MEDICAL_AGI_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import json
import base64
import os
import re
import torch
from transformers import pipeline
from openai import OpenAI
from google.colab import userdata
from datetime import datetime

# --- CONCEPTUAL LeJEPA MODULE START ---

# Function to conceptually represent the LeJEPA call and feature compression
def LeJEPA_Feature_Extractor(client, image_url):
    """
    Pillar 1: Simulates the Analog -> Digital conversion.
    The LeJEPA framework (Provably optimal JEPA loss) is used to learn
    highly stable, minimal risk representations, which feed into the classifier head.
    Returns a verifiable digital fact based on visual feature extraction.
    """
    try:
        # We use the Qwen API to simulate the Classifier Head's output (Analog -> Digital Bridge)
        prompt_for_fact_extraction = (
            "You are a clinical image classifier. Based on the provided CT images, output ONLY "
            "the raw, verifiable radiological signs (colon distention, mural thickening, fat stranding) "
            "as a single, factual sentence. Do not diagnose."
        )

        # Check if the client is the Mock client (meaning a live call will not work anyway)
        if hasattr(client, 'is_mock') or image_url is None:
             # UPDATED MOCK OUTPUT
             return "LeJEPA Feature Extraction Success (Simulated). Classifier Head Output: massive fecal burden, colonic distention, mural thickening, and fat stranding."

        # Use the Multimodal LLM to simulate the classification step on the Base64 image
        completion = client.chat.completions.create(
            model="qwen/qwen3-vl-8b-thinking",
            messages=[
                {"role": "user", "content": [
                    {"type": "text", "text": prompt_for_fact_extraction},
                    {"type": "image_url", "image_url": {"url": image_url}}
                ]}
            ],
            timeout=30
        )
        return completion.choices[0].message.content.strip()

    except Exception as e:
        # UPDATED FALLBACK OUTPUT
        return f"LeJEPA Execution Failed ({e}). Defaulting to: massive fecal burden, mural thickening, and fat stranding."

# --- CONCEPTUAL LeJEPA MODULE END ---


# --- 1. Client Setup and Utilities (Base64 FIX) ---\n
# Client Setup (using OpenAI SDK pointing to OpenRouter API)
try:
    OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
    if not OPENROUTER_API_KEY:
        raise ValueError("OPENROUTER_API_KEY is not set in Colab Secrets.")
    qwen_client = OpenAI (
        base_url="https://openrouter.ai/api/v1",
        api_key=OPENROUTER_API_KEY,
    )
except Exception as e:
    # Mock client logic for local execution/testing
    class MockClient:
        def __init__(self):
            self.iteration = 0
            self.is_mock = True # Add flag to identify mock client

        def chat(self): return self
        def completions(self): return self
        def create(self, **kwargs):
            self.iteration += 1
            if self.iteration == 1:
                 # Initial output is intentionally incomplete
                 mock_content = "### Most Likely Diagnosis: Fecal Impaction with Secondary Colitis. Acute Management: Enemas. Long-Term: Physical Therapy."
            elif self.iteration == 2:
                 # Output after first refinement (closer to target)
                 mock_content = "### Most Likely Diagnosis: Stercoral Colitis. Acute Management: Endoscopic removal (flexible sigmoidoscopy). Long-Term: Physical Therapy. Potential Complications: Necrosis."
            else:
                 # Converged output (meeting all validation checks)
                 mock_content = f"### Most Likely Diagnosis: Stercoral Colitis. Acute Management: Endoscopic removal (flexible sigmoidoscopy). Long-Term: Pelvic-Floor Physical Therapy, Anorectal Manometry, and Biofeedback. Potential Complications: Focal-Pressure Necrosis and Perforation."

            if self.iteration >= 5:
                 return None

            mock_choice = type('MockChoice', (object,), {'message': type('MockMessage', (object,), {'content': mock_content})()})
            return type('MockResponse', (object,), {
                'choices': [mock_choice]
            })()

    qwen_client = MockClient()
    print(f"Using Mock Client due to configuration error: {e}. Output will be simulated.")

# Utility Function: Convert Local Image to Base64 (FIXED)
def image_to_base64(file_path):
    """
    Reads local file path and converts it to a Base64 data URI.
    Returns None on failure to prevent passing a malformed URL to the API.
    """
    try:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Image file not found at {file_path}")

        with open(file_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

        if not encoded_string:
            raise ValueError("Base64 encoding resulted in an empty string.")

        return f"data:image/png;base64,{encoded_string}"

    except Exception as e:
        # Return None on genuine file/encoding failure
        print(f"[BASE64 ERROR] File/Encoding failed: {e}")
        return None

# --- 2. Agent Class Definitions ---

class ImageAnalysisAgent:
    """Pillar 1: Perception/Fact Grounding. The wrapper for the LeJEPA pipeline."""
    def analyze_image(self, client, image_url):
        # Delegate feature extraction and classification to the dedicated module
        return LeJEPA_Feature_Extractor(client, image_url)


class ValidationAgent:
    """Pillars 3 & 4: Constraint and Safety Enforcement"""
    def __init__(self):
        self.expected_patterns = {
            "diagnosis": [r"stercoral\s+colitis"],
            "acute_procedure": [r"endoscopic\s+(?:removal|disimpaction|evacuation|sigmoidoscopy)"],
            "long_term_dx": [r"anorectal\s+manometry", r"puborectalis\s+dysfunction"],
            "long_term_tx": [r"pelvic-floor\s+physical\s+therapy", r"biofeedback"],
            "complications": [r"perforation", r"necrosis", r"ischemia"]
        }

    def validate_output(self, model_output):
        issues = []
        lower_output = model_output.lower()

        if not any(re.search(pattern, lower_output) for pattern in self.expected_patterns["diagnosis"]):
            issues.append("Diagnosis is missing the precise medical term 'Stercoral Colitis'.")

        if not any(re.search(pattern, lower_output) for pattern in self.expected_patterns["acute_procedure"]):
            issues.append("The acute intervention plan lacks specific mention of endoscopic methods for evacuation.")

        if not any(re.search(pattern, lower_output) for pattern in self.expected_patterns["long_term_dx"]):
            issues.append("Long-term management is incomplete; assessment (e.g., anorectal manometry) is missing.")

        if not any(re.search(pattern, lower_output) for pattern in self.expected_patterns["long_term_tx"]):
            issues.append("Long-term therapy is incomplete; must explicitly include 'pelvic-floor physical therapy' or 'biofeedback.'")

        if not any(re.search(pattern, lower_output) for pattern in self.expected_patterns["complications"]):
            issues.append("Potential complications such as necrosis or perforation are not explicitly noted.")

        return issues if issues else ["Output aligns with expected clinical criteria."]

class PromptEngineerAgent:
    """Pillar 5: Adaptive Steering and Workflow Management"""
    def __init__(self):
        self.base_prompt = (\
            "You are a radiologist. Analyze the provided multi-slice CT scan images (A, B, C) "\
            "of a 23-year-old male patient with a history of autism spectrum disorder and chronic constipation, "\
            "presenting with abdominal pain, nausea, and vomiting. Focus on findings in the pelvis and abdomen, "\
            "particularly any abnormalities related to the colon, and determine the **most likely diagnosis** "\
            "based on these observations and the patient's clinical background. Provide a detailed rationale "\
            "linking the imaging findings to the diagnosis, recommend appropriate follow-up imaging or interventions, "\
            "and include a differential diagnosis considering potential inflammatory, neoplastic, or obstructive causes. "\
            "Identify any potential complications suggested by the imaging and suggest long-term management strategies "\
            "for underlying conditions contributing to the patient's presentation, including assessment of pelvic floor "\
            "function with an anorectal manometry and pelvic-floor physical therapy. "\
        )
        self.iteration = 0

    def refine_prompt(self, image_findings, validation_feedback):
        self.iteration += 1
        refined_prompt = self.base_prompt

        # Embed the LeJEPA output (the grounded facts) into the reasoning prompt
        if image_findings:
            # UPDATED PRINT STATEMENT
            refined_prompt += f" The images show the following critical facts (from LeJEPA analysis): {image_findings}. Use these observations to guide your diagnosis and recommendations, considering the patient's chronic condition and acute symptoms, with a focus on impaction-related findings such as fecal impaction leading to ischemic inflammation."

        if validation_feedback and "Output aligns" not in validation_feedback[0]:
            refinement_instruction = f" **CRITICAL REFINEMENT (Iteration {self.iteration}):** The previous response must be corrected to meet the following specific clinical requirements: "

            if "Diagnosis is missing the precise medical term" in " ".join(validation_feedback):
                refinement_instruction += "The **Diagnosis** section MUST explicitly use the term **Stercoral Colitis**."
            if "acute intervention plan lacks specific mention" in " ".join(validation_feedback):
                refinement_instruction += "Ensure the **Acute Management** explicitly includes **endoscopic removal** (e.g., flexible sigmoidoscopy) for disimpaction."
            if "Long-term therapy is incomplete" in " ".join(validation_feedback):
                refinement_instruction += "Ensure the **Long-Term Management** explicitly includes **pelvic-floor physical therapy** or **biofeedback**."
            if "Potential complications such as necrosis or perforation are not explicitly noted." in " ".join(validation_feedback):
                 refinement_instruction += "Explicitly note the risk of **perforation/necrosis** as a complication."

            refined_prompt += refinement_instruction

        return refined_prompt

# --- 3. Main Execution Loop (The Orchestrator) ---

def run_multi_agent_experiment():
    print(f"\n--- Multi-Agent Experiment Started at {datetime.now().strftime('%I:%M %p %Z on %B %d, %Y')} ---")

    # Initialize agents
    prompt_engineer = PromptEngineerAgent()
    image_analyzer = ImageAnalysisAgent()
    validator = ValidationAgent()

    # NOTE: This path MUST be valid for live calls
    image_path = "/content/drive/MyDrive/datasetsmedical/image.png"
    image_url = image_to_base64(image_path)

    # Check if we are running in a state where a live call is possible but image failed.
    if image_url is None and not hasattr(qwen_client, 'is_mock'):
        print("\nFATAL ERROR: Image file could not be encoded. Cannot proceed with live multimodal call.")
        return

    # Step 1: ESTABLISH GROUNDED PERCEPTION FACTS (LeJEPA Layer Output)
    image_findings = image_analyzer.analyze_image(qwen_client, image_url)
    # UPDATED PRINT STATEMENT
    print(f"\n--- Grounded Perception Facts Established (LeJEPA Layer Output) ---")
    print(f"Fact Set: {image_findings}")

    # Step 2: Refine Prompt and Iterate (The AGI Loop)
    max_iterations = 5
    current_iteration = 0
    validation_result = []

    refined_prompt = prompt_engineer.refine_prompt(image_findings, validation_result)

    while current_iteration < max_iterations:
        print(f"\n--- Iteration {current_iteration + 1} ---")

        completion = None

        try:
            # Call the core Multimodal LLM (The Reasoning Engine)
            completion = qwen_client.chat.completions.create(
                model="qwen/qwen3-vl-8b-thinking",
                messages=[
                    {"role": "user", "content": [
                        {"type": "text", "text": refined_prompt},
                        {"type": "image_url", "image_url": {"url": image_url}}
                    ]}
                ],
                timeout=30
            )

            # API Response Guardrail
            if completion is None or not completion.choices or not completion.choices[0].message.content:
                print("\n[CRITICAL API ERROR] API returned an invalid/empty response. Breaking the loop to prevent crash.")
                break

            model_output = completion.choices[0].message.content
            print(f"Qwen3-VL Analysis Result: {model_output[:80] + '...'}")

            # Step 3: Validate Output (Constraint Check)
            validation_result = validator.validate_output(model_output)
            print(f"Validation Agent Feedback: {validation_result}")

            # Step 4: Check for Convergence
            if "Output aligns" in validation_result[0]:
                print("\n**AGENTS CONVERGED: Validation criteria met. Stopping iteration.**")
                break

            # If validation fails, prepare the prompt for the next loop
            refined_prompt = prompt_engineer.refine_prompt(image_findings, validation_result)
            current_iteration += 1

        except Exception as e:
            # Catch general connection/timeout errors
            print(f"\n[EXCEPTION CAUGHT] An error occurred during Qwen-VL inference (Iteration {current_iteration + 1}): {e}")
            break

    if current_iteration == max_iterations and (not completion or "Output aligns" not in validation_result[0]):
        print("\nMaximum iterations reached without convergence.")

# Execute the experiment
if __name__ == "__main__":
    run_multi_agent_experiment()


--- Multi-Agent Experiment Started at 10:45 PM  on November 13, 2025 ---

--- Grounded Perception Facts Established (LeJEPA Layer Output) ---
Fact Set: Colon distention, mural thickening, and fat stranding are observed.

--- Iteration 1 ---
Qwen3-VL Analysis Result: ### Radiologist Analysis: Multislice CT Scan of a 23-Year-Old Male with Acute Ab...
Validation Agent Feedback: ["Diagnosis is missing the precise medical term 'Stercoral Colitis'.", 'The acute intervention plan lacks specific mention of endoscopic methods for evacuation.']

--- Iteration 2 ---
Qwen3-VL Analysis Result: ### Comprehensive Radiological Analysis and Clinical Management Plan  

---

###...
Validation Agent Feedback: ['Output aligns with expected clinical criteria.']

**AGENTS CONVERGED: Validation criteria met. Stopping iteration.**
